In [ ]:
# %uv pip install torch==2.0.0+cpu --index-url https://download.pytorch.org/whl/cpu
%pip install torch-cluster torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
%pip install torch-geometric==2.3.1
%pip install matplotlib pandas scikit-learn tqdm numpy networkx

/home/ken/workspace/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.
/home/ken/workspace/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/ken/workspace/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/ken/workspace/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from torch_geometric.datasets import Planetoid

ModuleNotFoundError: No module named 'torch'

In [ ]:
# torch_geometric の Dataset としてダウンロード
dataset = Planetoid(root="./dataset", name="Cora", split="full")
print(f"データセットの型 : {type(dataset)}")
print(f"ネットワークの型 : {type(dataset._data)}")
print(f"ノード数 : {dataset.num_nodes}")
print(f"エッジ数 : {dataset.num_edges}")
print(f"特徴量の次元数 : {dataset.num_node_features}")
print(f"クラス数 : {dataset.num_classes}")
data = dataset[0]